<a href="https://colab.research.google.com/github/shrivatsashetty/Projects/blob/main/Smartphone_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SENTIMENT ANALYSIS PROJECT ON AMAMZON CUSTOMER REVIEWS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# nltk module for text processing
import nltk


# Item Table

In [ ]:
item_table = pd.read_csv("/content/drive/MyDrive/Data-Science/20191226-items.csv")

# Review Table

In [ ]:
review_table = pd.read_csv("/content/drive/MyDrive/Data-Science/20191226-reviews.csv")

# Checking Structure of each

In [ ]:
item_table.shape

(720, 10)

In [ ]:
item_table.isna().sum()

asin             0
brand            4
title            0
url              0
image            0
rating           0
reviewUrl        0
totalReviews     0
price            0
originalPrice    0
dtype: int64

In [ ]:
item_table = item_table.dropna()

In [ ]:
review_table.shape

(67986, 8)

In [ ]:
review_table.isna().sum()

asin                0
name                2
rating              0
date                0
verified            0
title              14
body               21
helpfulVotes    40771
dtype: int64

In [ ]:
review_table = review_table.drop('helpfulVotes', axis = 1)

In [ ]:
review_table = review_table.dropna()

In [ ]:
review_table.head(3)

,asin,name,rating,date,verified,title,body
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha..."


In [ ]:
item_table.head(3)

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.0
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,3.3,https://www.amazon.com/product-reviews/B001AO4OUC,21,0.00,0.0


In [ ]:
item_series = item_table['originalPrice'].value_counts()

In [ ]:
(item_series !=0 ).sum()

89

# Dropping unwanted columns

In [ ]:
review_table.columns

Index(['asin', 'name', 'rating', 'date', 'verified', 'title', 'body'], dtype='object')

In [ ]:
item_table.columns

Index(['asin', 'brand', 'title', 'url', 'image', 'rating', 'reviewUrl',
       'totalReviews', 'price', 'originalPrice'],
      dtype='object')

In [ ]:
item_table = item_table.drop(columns = ['url', 'image', 'reviewUrl'], axis = 1)

In [ ]:
common_occurrences = review_table[review_table['asin'].isin(item_table['asin'])]


In [ ]:
common_occurrences.head(5)

,asin,name,rating,date,verified,title,body
14,B0009N5L7K,Marcel Thomas,1,"March 5, 2016",True,Stupid phone,DON'T BUY OUT OF SERVICE
15,B0009N5L7K,William B.,4,"February 9, 2006",False,Exellent Service,I have been with nextel for nearly a year now ...
16,B0009N5L7K,K. Mcilhargey,5,"February 7, 2006",False,I love it,"I just got it and have to say its easy to use,..."
17,B0009N5L7K,Stephen Cahill,1,"December 20, 2016",True,Phones locked,1 star because the phones locked so I have to ...
18,B0009N5L7K,Mihir,5,"December 13, 2009",True,Excellent product,The product has been very good. I had used thi...


In [ ]:
review_table = common_occurrences

# Now the review table contains only the entries whose asin(id) is present in the item table

# Checking for null if any

In [ ]:
item_table.isna().sum()

asin             0
brand            0
title            0
rating           0
totalReviews     0
price            0
originalPrice    0
dtype: int64

In [ ]:
review_table.isna().sum()

asin        0
name        0
rating      0
date        0
verified    0
title       0
body        0
dtype: int64

# ***Feature Engineering the review data farme***

In [ ]:
required_fields = item_table[['asin', 'brand', 'title', 'rating', 'totalReviews', 'price']]

In [ ]:
required_fields

,asin,brand,title,rating,totalReviews,price
1,B0009N5L7K,Motorola,Motorola I265 phone,3.0,7,49.95
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,2.7,22,99.99
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,3.3,21,0.00
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,3.1,12,149.99
5,B001GQ3DJM,Nokia,Nokia 1680 Black Phone (T-Mobile),2.7,3,0.00
...,...,...,...,...,...,...
715,B07ZPKZSSC,Apple,"Apple iPhone 11 Pro, 64GB, Fully Unlocked - Sp...",1.0,1,949.00
716,B07ZQSGP53,Xiaomi,"Xiaomi Redmi Note 8, 32GB/3GB RAM 6.3"" FHD+ Di...",4.6,3,150.96
717,B081H6STQQ,Sony,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,4.5,70,948.00
718,B081TJFVCJ,Apple,"Apple iPhone X, 64GB, Gray - Fully Unlocked (R...",5.0,1,478.97


In [ ]:
required_fields.shape

(716, 6)

In [ ]:
required_fields.columns

Index(['asin', 'brand', 'title', 'rating', 'totalReviews', 'price'], dtype='object')

In [ ]:
review_table.columns

Index(['asin', 'name', 'rating', 'date', 'verified', 'title', 'body'], dtype='object')

In [ ]:
def add_fields(row):
    asin = row['asin']
    # matching based on asin
    matching_product = required_fields[required_fields['asin'] == asin]

    if not matching_product.empty:
        row['brand'] = matching_product['brand'].iloc[0]
        row['title'] = matching_product['title'].iloc[0]
        row['rating'] = matching_product['rating'].iloc[0]
        row['totalReviews'] = matching_product['totalReviews'].iloc[0]
        row['price'] = matching_product['price'].iloc[0]

    return row

# Apply the function to each row in 'review_table' to add 'brand' and 'title'
review_table = review_table.apply(add_fields, axis=1)





In [ ]:
# Display the updated 'review_table' DataFrame
review_table.head(5)

,asin,name,rating,date,verified,title,body,brand,totalReviews,price
14,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,49.95
15,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,49.95
16,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,49.95
17,B0009N5L7K,Stephen Cahill,3.0,"December 20, 2016",True,Motorola I265 phone,1 star because the phones locked so I have to ...,Motorola,7,49.95
18,B0009N5L7K,Mihir,3.0,"December 13, 2009",True,Motorola I265 phone,The product has been very good. I had used thi...,Motorola,7,49.95


# Lets check the shape of the data frame to check integrity

In [ ]:
review_table.shape

(67754, 10)

# All rows are retained which is good

# Now correcting the index

In [ ]:
review_table = review_table.reset_index(drop=True)

In [ ]:
review_table

,asin,name,rating,date,verified,title,body,brand,totalReviews,price
0,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,49.95
1,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,49.95
2,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,49.95
3,B0009N5L7K,Stephen Cahill,3.0,"December 20, 2016",True,Motorola I265 phone,1 star because the phones locked so I have to ...,Motorola,7,49.95
4,B0009N5L7K,Mihir,3.0,"December 13, 2009",True,Motorola I265 phone,The product has been very good. I had used thi...,Motorola,7,49.95
...,...,...,...,...,...,...,...,...,...,...
67749,B081H6STQQ,jande,4.5,"August 16, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,I love the camera on this phone. The screen is...,Sony,70,948.00
67750,B081H6STQQ,2cool4u,4.5,"September 14, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,I've been an Xperia user for several years and...,Sony,70,948.00
67751,B081H6STQQ,simon,4.5,"July 14, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,buy one more for my cousin,Sony,70,948.00
67752,B081TJFVCJ,Tobiasz Jedrysiak,5.0,"December 24, 2019",True,"Apple iPhone X, 64GB, Gray - Fully Unlocked (R...",Product looks and works like new. Very much re...,Apple,1,478.97


# Renaming the columns headings

In [ ]:
review_table.describe()

,rating,totalReviews,price
count,67754.000000,67754.000000,67754.000000
mean,3.766637,374.559937,221.468162
std,0.429637,262.497030,188.573342
min,1.000000,1.000000,0.000000
25%,3.500000,153.000000,103.980000
50%,3.800000,336.000000,179.990000
75%,4.100000,558.000000,300.550000
max,5.000000,983.000000,999.990000


In [ ]:
review_table.tail(5)

,asin,name,rating,date,verified,title,body,brand,totalReviews,price
67749,B081H6STQQ,jande,4.5,"August 16, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,I love the camera on this phone. The screen is...,Sony,70,948.00
67750,B081H6STQQ,2cool4u,4.5,"September 14, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,I've been an Xperia user for several years and...,Sony,70,948.00
67751,B081H6STQQ,simon,4.5,"July 14, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,buy one more for my cousin,Sony,70,948.00
67752,B081TJFVCJ,Tobiasz Jedrysiak,5.0,"December 24, 2019",True,"Apple iPhone X, 64GB, Gray - Fully Unlocked (R...",Product looks and works like new. Very much re...,Apple,1,478.97
67753,B0825BB7SG,Owen Gonzalez,5.0,"December 11, 2019",False,Straight Talk Samsung Galaxy A10e Smartphone 5...,I love the size and style of this phone. It is...,Samsung,1,139.00


# converting the prices to normal format

In [ ]:
len(review_table)

67754

In [ ]:
i = len(review_table)
for cost in review_table['price']:
  if i == 0 : # if i is 0 then i-1 will become negative
    break
  review_table['price'][i-1] = review_table['price'][i-1] * 100
  i -= 1


<ipython-input-37-aed9c7cf665c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_table['price'][i-1] = review_table['price'][i-1] * 100


In [ ]:
review_table['price'].tail(3)

67751    94800.0
67752    47897.0
67753    13900.0
Name: price, dtype: float64

In [ ]:
review_table['price'].head(3)

0    4995.0
1    4995.0
2    4995.0
Name: price, dtype: float64

# Renaming the table headings into readble names

In [ ]:
review_table.head(4)

,asin,name,rating,date,verified,title,body,brand,totalReviews,price
0,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,4995.0
1,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,4995.0
2,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,4995.0
3,B0009N5L7K,Stephen Cahill,3.0,"December 20, 2016",True,Motorola I265 phone,1 star because the phones locked so I have to ...,Motorola,7,4995.0


In [ ]:
review_table.columns

Index(['asin', 'name', 'rating', 'date', 'verified', 'title', 'body', 'brand',
       'totalReviews', 'price'],
      dtype='object')

In [ ]:
review_table = review_table.rename(columns = {'asin' : 'product_id', 'name' : 'reviewer_name', 'rating' : 'star_ratings', 'date':'review_date', 'title': 'device_name',
                               'body':'reviews', 'totalReviews' : 'total_reviews'})

In [ ]:
review_table.head(3)

,product_id,reviewer_name,star_ratings,review_date,verified,device_name,reviews,brand,total_reviews,price
0,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,4995.0
1,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,4995.0
2,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,4995.0


# Headings changed

In [ ]:
# importing regular expression moudle
import re

def remove_special_characters(text):
    # Defining a regular expression pattern to match special characters
    pattern = r'[^a-zA-Z0-9\s]'

    # Use the sub() method to replace matched special characters with an empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

# cleaning the text reviews off the special caharacters
* The special characters cause errors while we run SIA on them

# Lets define a function to filter special characters

In [ ]:
import re

def remove_special_characters(text):
    # Define a regular expression pattern to match special characters
    pattern = r'[^a-zA-Z0-9\s]'

    # Use the sub() method to replace matched special characters with an empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

# Example usage:
input_text = "Hello, this is a sample text with $pecial characters! 12345."
cleaned_text = remove_special_characters(input_text)
print(cleaned_text)

Hello this is a sample text with pecial characters 12345


In [ ]:
# Apply the remove_special_characters function to the 'review' column and store the results in 'cleaned_reviews'
review_table['cleaned_reviews'] = review_table['reviews'].apply(remove_special_characters)

In [ ]:
review_table.head(6)

,product_id,reviewer_name,star_ratings,review_date,verified,device_name,reviews,brand,total_reviews,price,cleaned_reviews
0,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,4995.0,DONT BUY OUT OF SERVICE
1,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,4995.0,I have been with nextel for nearly a year now ...
2,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,4995.0,I just got it and have to say its easy to use ...
3,B0009N5L7K,Stephen Cahill,3.0,"December 20, 2016",True,Motorola I265 phone,1 star because the phones locked so I have to ...,Motorola,7,4995.0,1 star because the phones locked so I have to ...
4,B0009N5L7K,Mihir,3.0,"December 13, 2009",True,Motorola I265 phone,The product has been very good. I had used thi...,Motorola,7,4995.0,The product has been very good I had used this...
5,B0009N5L7K,L. Hughes,3.0,"July 21, 2005",False,Motorola I265 phone,My problems with nextel did not stop when I ca...,Motorola,7,4995.0,My problems with nextel did not stop when I ca...


# Done with pre processing the data frame

# The VADER approach

**We will use NLTK's SentimentIntensityAnalyzer to get the negative, neutral, positive scores of the text**


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# creating an object for SIA
sia  = SentimentIntensityAnalyzer()

# Few samples of sentiment intensity analyzer

In [ ]:
sia.polarity_scores("RVCE")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [ ]:
sia.polarity_scores("I Like RVCE")

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}

In [ ]:
sia.polarity_scores("I love RVCE !!!")

{'neg': 0.0, 'neu': 0.283, 'pos': 0.717, 'compound': 0.7249}

In [ ]:
sia.polarity_scores("Subject")

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [ ]:
sia.polarity_scores("I hate this Subject")

{'neg': 0.649, 'neu': 0.351, 'pos': 0.0, 'compound': -0.5719}

In [ ]:
top_reviews = review_table.head(5)

In [ ]:
for review in top_reviews['reviews']:
  print(review)

DON'T BUY OUT OF SERVICE
I have been with nextel for nearly a year now I started out this time last year with the Motorola i205 and just upgraded to the i265 it is one of the best phones I have ever had the service is the best I have ever had I have no problems making or reciving calls. If you are considering nextel give it a shot they are in my opinion the best cell company out there.
I just got it and have to say its easy to use, i can hear the person talking just fine and i have had no problems dealing with nextel.
1 star because the phones locked so I have to pay additional fees to unlock it
The product has been very good. I had used this cell phone in one of my projects and it worked wonders. I will definitely recommend this to anyone interested in buying it....


In [ ]:
len(review_table)

67754

# Adding a new column called sl_no

In [ ]:
review_table['sl_no'] = range(1, len(review_table) + 1)


In [ ]:
review_table.head(4)

,product_id,reviewer_name,star_ratings,review_date,verified,device_name,reviews,brand,total_reviews,price,cleaned_reviews,sl_no
0,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,4995.0,DONT BUY OUT OF SERVICE,1
1,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,4995.0,I have been with nextel for nearly a year now ...,2
2,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,4995.0,I just got it and have to say its easy to use ...,3
3,B0009N5L7K,Stephen Cahill,3.0,"December 20, 2016",True,Motorola I265 phone,1 star because the phones locked so I have to ...,Motorola,7,4995.0,1 star because the phones locked so I have to ...,4


In [ ]:
review_table.tail(4)

,product_id,reviewer_name,star_ratings,review_date,verified,device_name,reviews,brand,total_reviews,price,cleaned_reviews,sl_no
67750,B081H6STQQ,2cool4u,4.5,"September 14, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,I've been an Xperia user for several years and...,Sony,70,94800.0,Ive been an Xperia user for several years and ...,67751
67751,B081H6STQQ,simon,4.5,"July 14, 2019",False,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,buy one more for my cousin,Sony,70,94800.0,buy one more for my cousin,67752
67752,B081TJFVCJ,Tobiasz Jedrysiak,5.0,"December 24, 2019",True,"Apple iPhone X, 64GB, Gray - Fully Unlocked (R...",Product looks and works like new. Very much re...,Apple,1,47897.0,Product looks and works like new Very much rec...,67753
67753,B0825BB7SG,Owen Gonzalez,5.0,"December 11, 2019",False,Straight Talk Samsung Galaxy A10e Smartphone 5...,I love the size and style of this phone. It is...,Samsung,1,13900.0,I love the size and style of this phone It is ...,67754


# Creating one more data frame for storing polarity scores

In [ ]:
# creating a dictionary to store the results of polarity scores
polarity_score_results = {}
i = 1
for cleaned_review in review_table['cleaned_reviews']:
  if i == len(review_table) + 1:
    break
  polarity_score_results[i] = sia.polarity_scores(cleaned_review)
  i += 1

In [ ]:
polarity_score_results[len(review_table)]

{'neg': 0.0, 'neu': 0.585, 'pos': 0.415, 'compound': 0.9485}

In [ ]:
try:
  polarity_score_results[len(review_table) +1 ]
except Exception as e:
  print(e)
  print("Exception occoured and handled")



67755
Exception occoured and handled


In [ ]:
polarity_scores_table = pd.DataFrame(polarity_score_results).T

In [ ]:
polarity_scores_table

,neg,neu,pos,compound
1,0.000,1.000,0.000,0.0000
2,0.061,0.783,0.157,0.8658
3,0.155,0.696,0.149,-0.0516
4,0.113,0.887,0.000,-0.1689
5,0.000,0.701,0.299,0.8777
...,...,...,...,...
67750,0.051,0.830,0.119,0.8779
67751,0.079,0.807,0.114,0.7729
67752,0.000,1.000,0.000,0.0000
67753,0.000,0.570,0.430,0.6461


In [ ]:
polarity_scores_table.columns

Index(['neg', 'neu', 'pos', 'compound'], dtype='object')

In [ ]:
review_table.shape

(67754, 12)

In [ ]:
polarity_scores_table = polarity_scores_table.reset_index().rename(columns={"index" : "sl_no"})

In [ ]:
polarity_scores_table

,sl_no,neg,neu,pos,compound
0,1,0.000,1.000,0.000,0.0000
1,2,0.061,0.783,0.157,0.8658
2,3,0.155,0.696,0.149,-0.0516
3,4,0.113,0.887,0.000,-0.1689
4,5,0.000,0.701,0.299,0.8777
...,...,...,...,...,...
67749,67750,0.051,0.830,0.119,0.8779
67750,67751,0.079,0.807,0.114,0.7729
67751,67752,0.000,1.000,0.000,0.0000
67752,67753,0.000,0.570,0.430,0.6461


In [ ]:
polarity_scores_table.columns

Index(['sl_no', 'neg', 'neu', 'pos', 'compound'], dtype='object')

In [ ]:
review_table = polarity_scores_table.merge(review_table, how = "left", on = "sl_no") # it will merge based on common column

In [ ]:
review_table.head(3)

,sl_no,neg,neu,pos,compound,product_id,reviewer_name,star_ratings,review_date,verified,device_name,reviews,brand,total_reviews,price,cleaned_reviews
0,1,0.000,1.000,0.000,0.0000,B0009N5L7K,Marcel Thomas,3.0,"March 5, 2016",True,Motorola I265 phone,DON'T BUY OUT OF SERVICE,Motorola,7,4995.0,DONT BUY OUT OF SERVICE
1,2,0.061,0.783,0.157,0.8658,B0009N5L7K,William B.,3.0,"February 9, 2006",False,Motorola I265 phone,I have been with nextel for nearly a year now ...,Motorola,7,4995.0,I have been with nextel for nearly a year now ...
2,3,0.155,0.696,0.149,-0.0516,B0009N5L7K,K. Mcilhargey,3.0,"February 7, 2006",False,Motorola I265 phone,"I just got it and have to say its easy to use,...",Motorola,7,4995.0,I just got it and have to say its easy to use ...


# The results need to be improved